In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb

In [5]:
csv = pd.read_csv('../lie_detector/datasets/raw/TrialData/Annotation/All_Gestures_Deceptive and Truthful.csv')

In [8]:
csv.columns

Index(['id', 'OtherGestures', 'Smile', 'Laugh', 'Scowl',
       'otherEyebrowMovement', 'Frown', 'Raise', 'OtherEyeMovements',
       'Close-R', 'X-Open', 'Close-BE', 'gazeInterlocutor', 'gazeDown',
       'gazeUp', 'otherGaze', 'gazeSide', 'openMouth', 'closeMouth',
       'lipsDown', 'lipsUp', 'lipsRetracted', 'lipsProtruded', 'SideTurn',
       'downR', 'sideTilt', 'backHead', 'otherHeadM', 'sideTurnR', 'sideTiltR',
       'waggle', 'forwardHead', 'downRHead', 'singleHand', 'bothHands',
       'otherHandM', 'complexHandM', 'sidewaysHand', 'downHands', 'upHands',
       'class'],
      dtype='object')

In [9]:
y = csv['class'].map(lambda x: 1 if x == 'deceptive' else 0)
X = csv.drop(['id', 'class'], axis=1)

In [10]:
X.head()

,OtherGestures,Smile,Laugh,Scowl,otherEyebrowMovement,Frown,Raise,OtherEyeMovements,Close-R,X-Open,...,waggle,forwardHead,downRHead,singleHand,bothHands,otherHandM,complexHandM,sidewaysHand,downHands,upHands
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
2,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
4,1,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [14]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64

In [227]:
indices = np.arange(len(X))
np.random.shuffle(indices)

trn_indices = indices[:100]
val_indices = indices[100:]

In [228]:
X_trn = X.iloc[trn_indices]
X_val = X.iloc[val_indices]
y_trn = y.iloc[trn_indices]
y_val = y.iloc[val_indices]

dat_trn = lgb.Dataset(X_trn, label=y_trn)
dat_val = lgb.Dataset(X_val, label=y_val)

In [262]:
params = {
    'learning_rate': 0.05,
#     'max_depth': -1, # smaller max depth for less overfitting
    'num_leaves': 2, # smaller num leaves for less overfitting
#     'min_data_in_leaf': 1, # larger min data for less overfitting
#     'max_bin': 2, # smaller max bin for less overfitting
    'num_iteration': 1000,
#     'feature_fraction': 0.4, # smaller for less overfitting
    'bagging_fraction': 0.7, # smaller for less overfitting
    'bagging_freq': 5, # smaller for less overfitting
#     'lambda_l1': .01, # larger for less overfitting
#     'lambda_l2': .02, # larger for less overfitting
    'objective': 'binary',
}

model = lgb.train(params, dat_trn, valid_sets=[dat_trn, dat_val], verbose_eval=100, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.479772	valid_1's binary_logloss: 0.538795
[200]	training's binary_logloss: 0.426811	valid_1's binary_logloss: 0.49177
Early stopping, best iteration is:
[185]	training's binary_logloss: 0.432452	valid_1's binary_logloss: 0.476792


In [263]:
y_val_hat = model.predict(X_val)

In [264]:
y_val_hat

array([0.68493514, 0.69795885, 0.34361657, 0.59493365, 0.50598622,
       0.96993056, 0.7376434 , 0.42923476, 0.19952578, 0.21841405,
       0.64196123, 0.29837696, 0.58943416, 0.57579002, 0.2748535 ,
       0.61789136, 0.96626833, 0.3596406 , 0.85286263, 0.46616205,
       0.65260131])

In [265]:
(1-sum(abs(y_val_hat.round() - y_val)) / len(y_val))

0.7619047619047619

In [266]:
y_val.values.astype(np.float)

array([1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0.])

In [267]:
y_val_hat.round()

array([1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1.,
       0., 1., 0., 1.])